In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from sklearn import svm
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from collections import defaultdict
import networkx as nx
import gzip
import math
import nltk
import string
from nltk.stem.porter import *
import time
import datetime

def parseDataFromURL(fname):
    for l in urlopen(fname):
        yield eval(l)

def parseData(fname):
    for l in open(fname):
        yield eval(l)
    
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)


In [ ]:

data = []

for d in readGz("/content/drive/MyDrive/assignment1/train_Category.json.gz"):
    data.append(d)

Ntrain = (9*len(data))//10
# dataTrain = data[:Ntrain]



In [ ]:

data_test = []

for d in readGz("/content/drive/MyDrive/assignment1/test_Category.json.gz"):
    data_test.append(d)



In [ ]:

data[0]



{'date': '2014-02-07',
 'early_access': False,
 'genre': 'Adventure',
 'genreID': 3,
 'hours': 4.1,
 'reviewID': 'r75487422',
 'text': 'Short Review:\nA good starting chapter for this series, despite the main character being annoying (for now) and a short length. The story is good and actually gets more interesting. Worth the try.\nLong Review:\nBlackwell Legacy is the first on the series of (supposedly) 5 games that talks about the main protagonist, Rosangela Blackwell, as being a so called Medium, and in this first chapter we get to know how her story will start and how she will meet her adventure companion Joey...and really, that\'s really all for for now and that\'s not a bad thing, because in a way this game wants to show how hard her new job is, and that she cannot escape her destiny as a medium.\nMy biggest complain for this chapter, except the short length, it\'s the main protagonist being a "bit" too annoying to be likeable, and most of her dialogues will always be about compl

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
sp = set(string.punctuation)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
all_words = [nltk.word_tokenize(d['text']) for d in data]
plot = []

# Removing Stop Words
from nltk.corpus import stopwords
for i in range(len(all_words)):
  all_words[i] = [w.lower() for w in all_words[i] if w.lower() not in stopwords.words('english')]
  string = ' '.join(all_words[i])
  r = ''.join([c for c in string if not c in sp])
  plot.append(r)

In [ ]:
genre_new = [[d['genreID']] for d in data]
plot[:10]

['short review  good starting chapter series  despite main character annoying   short length  story good actually gets interesting  worth try  long review  blackwell legacy first series  supposedly  5 games talks main protagonist  rosangela blackwell  called medium  first chapter get know story start meet adventure companion joey  really  s really s bad thing  way game wants show hard new job  escape destiny medium  biggest complain chapter  except short length  s main protagonist  bit  annoying likeable  dialogues always complaining annoyed  understandable  sure  lighten  ya   however  considering next installments much likeable kind interesting  d say give shot see like  hate first game  might like next  always stop  recommend ',
 'think top 3 favirote tcgs time  wish added draw power make fact 75 card decks min 25 land makes mana screw happen quite often ',
 '10  suck 20  skill 15  concentrated power 5  pleasure 50  pain 0  reason purchase game',
 'game realy realy nice play  s hide

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(genre_new)

# transform target variable
y = multilabel_binarizer.transform(genre_new)
list(multilabel_binarizer.classes_)

[0, 1, 2, 3, 4]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3))
xtrain, xval, ytrain, yval = train_test_split(plot, y, test_size=0.2, shuffle=False)
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [ ]:
ytrainNew = []
for y_arr in ytrain:
  if 1 in list(y_arr):
    ytrainNew.append(list(y_arr).index(1))
  else:
    ytrainNew.append(0)


yvalNew = []
for y_arr in yval:
  if 1 in list(y_arr):
    yvalNew.append(list(y_arr).index(1))
  else:
    yvalNew.append(0)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
text_clf = Pipeline([('tfidf', TfidfVectorizer(max_features=30000)),
                      ('clf', LinearSVC()),
                    ])
text_clf.fit(xtrain, ytrainNew)

predictions = text_clf.predict(xval)

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(accuracy_score(yvalNew, predictions))

0.7558857142857143


In [ ]:
def feat(text):
  arr = [w.lower() for w in nltk.word_tokenize(text) if w.lower() not in stopwords.words('english')]
  string = ' '.join(arr)
  r = ''.join([c for c in string if not c in sp])
  return r

Xtest = [feat(d['text']) for d in data_test]




In [ ]:
#@title
xtest_tfidf = tfidf_vectorizer.transform(Xtest)
y_pred_test = clf.predict(xtest_tfidf)

In [ ]:
#@title
y_pred_prob = clf.predict_proba(xtest_tfidf)
y_pred_test = (y_pred_prob >= 0.36).astype(int)

In [ ]:
#@title
pred_test = []
counter = 0
for y_arr in y_pred_test:
  if 1 in list(y_arr):
    pred_test.append(list(y_arr).index(1))
  else:
    counter+=1
    pred_test.append(0)
counter/len(pred_test)

0.0349

In [ ]:
#@title
pred_test[:10]

[3, 1, 0, 0, 3, 0, 0, 0, 3, 0]

In [ ]:
len(ytrainNew+yvalNew)

175000

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
text_clf = Pipeline([('tfidf', TfidfVectorizer(max_features=30000)), ('clf', LinearSVC())])
text_clf.fit(Xtrain, ytrainNew)
yPredictions = text_clf.predict(Xtest)


In [ ]:

predictions = open("/content/drive/MyDrive/assignment1/predictions_Category_Aditya_A1.csv", 'w')
pos = 0
for l in open("/content/drive/MyDrive/assignment1/pairs_Category.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    _ = predictions.write(u + '-' + b + ',' + str(predic[pos]) + '\n')
    pos += 1

predictions.close()



In [ ]:
#@title
lr = LogisticRegression(C=8, max_iter=12000000)
clf = OneVsRestClassifier(lr)


# fit model on train data
clf.fit(xtrain_tfidf, ytrain)


# make predictions for validation set
y_pred = clf.predict(xval_tfidf)

f1_score(yval, y_pred, average="micro")

# Cs = [2,4,6,8]
# for c in Cs:
#   print(c)
#   lr = LogisticRegression(C=c, max_iter=12000000)
#   clf = OneVsRestClassifier(lr)


#   # fit model on train data
#   clf.fit(xtrain_tfidf, ytrain)


#   # make predictions for validation set
#   y_pred = clf.predict(xval_tfidf)

#   print(f1_score(yval, y_pred, average="micro"))

0.7414306551598394

In [ ]:
#@title
# predict probabilities
y_pred_prob = clf.predict_proba(xval_tfidf)
# y_pred_test = (y_pred_prob >= 0.36).astype(int)
# f1_score(yval, y_pred_test, average="micro")
best = []
thresholds = [x/100 for x in range(1,101)]
for t in thresholds:
#t = 0.3 # threshold value
  y_pred_new = (y_pred_prob >= t).astype(int)
  # evaluate performance
  best.append((f1_score(yval, y_pred_new, average="micro"),t))

best.sort()
best.reverse()
best

[(0.7526990411208918, 0.36),
 (0.7525250384935938, 0.37),
 (0.7522727272727273, 0.38),
 (0.7521281643361524, 0.35),
 (0.7521135610647778, 0.39),
 (0.7516860481252254, 0.34),
 (0.7515045315484825, 0.4),
 (0.7513248906152981, 0.41),
 (0.7512306780105072, 0.33),
 (0.750599618984965, 0.32),
 (0.7505734777432563, 0.42),
 (0.7498759160316487, 0.43),
 (0.7496868362289637, 0.31),
 (0.7487150870803841, 0.44),
 (0.7483530605343254, 0.3),
 (0.7478073413460118, 0.45),
 (0.7475094653741843, 0.29),
 (0.7465926299848561, 0.46),
 (0.7456442871859184, 0.47),
 (0.7455909993734753, 0.28),
 (0.7438374066785838, 0.48),
 (0.7438359788359789, 0.27),
 (0.7425041161276679, 0.49),
 (0.7419160104986878, 0.26),
 (0.7414306551598394, 0.5),
 (0.7399172025932773, 0.25),
 (0.7390798972225621, 0.51),
 (0.7374208751097353, 0.52),
 (0.7369872546619071, 0.24),
 (0.7356749055318095, 0.53),
 (0.7342612693142636, 0.23),
 (0.7335370416562733, 0.54),
 (0.7313168609797426, 0.22),
 (0.7304561028510358, 0.55),
 (0.72840580168609

In [ ]:
#@title
yval[:10]

array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])